## Chat tools

Sample notebook to showcase chat usage with tools.

Required imports:

In [1]:
# Copyright (c) 2024 Microsoft Corporation.

import logging
import os

from fnllm import LLMEventsLogger, LLMTool
from fnllm.openai import AzureOpenAIConfig, OpenAIChatRole, create_openai_chat_llm
from pydantic import Field

Example logger to showcase event logging:

In [2]:
def _create_logger():
    logger = logging.getLogger("CHAT")
    logger.setLevel(logging.DEBUG)

    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(
        logging.Formatter("[%(levelname)s %(asctime)s %(name)s] - %(message)s")
    )
    logger.handlers = [handler]

    return logger

Creating chat client:

In [3]:
config = AzureOpenAIConfig(
    endpoint=os.environ("OPENAI_API_ENDPOINT"),
    api_version=os.environ("OPENAI_API_VERSION"),
    model=os.environ("OPENAI_API_MODEL"),
    encoding=os.environ("OPENAI_ENCODING"),
)

chat_client = create_openai_chat_llm(config, events=LLMEventsLogger(_create_logger()))

Creating tools. The information sent to the LLM will be:

- **tool name**: from the class name or overwritten by `__tool_name__`
- **tool description**: from the class doc string or overwritten by `__tool_description__`
- **parameters**: the attributes with proper types defined in the class (including default values)
- **parameter descriptions**: derived from the `Field` description

In [4]:
class WeatherForecastTool(LLMTool):
    """Call this to get the weather forecast for a particular location."""

    location: str = Field(description="Location to get the weather forecast for.")

    date: str | None = Field(
        default=None,
        description="If passed, will provide the weather forecast for a previous time period.",
    )

    async def execute(self) -> str:
        """Execute the tool."""
        return f"Weather for {self.location}({self.date or 'now'}) - nice weather."


class AddNumbersTool(LLMTool):
    """Call this to sum a list of number."""

    # by default, will be AddNumbersTool
    __tool_name__ = "add_numbers"

    # by default, will be the class doc string
    __tool_description__ = "Use this to sum a list of number."

    numbers: list[float | int] = Field(description="List of number to sum.")

    async def execute(self) -> float | int:
        """Execute the tool."""
        return sum(self.numbers)

Calling the LLM:

In [5]:
response_0 = await chat_client(
    "How is the weather in Redmond, US? And what's 2 + 2 + 1",
    tools=[
        WeatherForecastTool,
        AddNumbersTool,
    ],
)

[INFO 2024-08-08 09:26:46,432 CHAT] - limit acquired for request, request_tokens=245, post_request_tokens=0
[INFO 2024-08-08 09:26:50,311 CHAT] - LLM usage with 200 total tokens (input=146, output=54)
[INFO 2024-08-08 09:26:50,312 CHAT] - limit released for request, request_tokens=245, post_request_tokens=0
[INFO 2024-08-08 09:26:50,313 CHAT] - request succeed with 0 retries in 3.90s and used 200 tokens


In [6]:
print(response_0.model_dump_json(indent=2))

{
  "output": {
    "raw_input": {
      "content": "How is the weather in Redmond, US? And what's 2 + 2 + 1",
      "role": "user"
    },
    "raw_output": {
      "content": null,
      "role": "assistant",
      "function_call": null,
      "tool_calls": [
        {
          "id": "call_Y0RsmenHwmwo19qM7xngaVEY",
          "function": {
            "arguments": "{\"location\": \"Redmond, US\"}",
            "name": "WeatherForecastTool"
          },
          "type": "function"
        },
        {
          "id": "call_CDSTpjUMG6TFI7Uyn9e2o5z1",
          "function": {
            "arguments": "{\"numbers\": [2, 2, 1]}",
            "name": "add_numbers"
          },
          "type": "function"
        }
      ]
    },
    "content": null,
    "usage": {
      "input_tokens": 146,
      "output_tokens": 54,
      "total_tokens": 200
    }
  },
  "raw_json": null,
  "parsed_json": null,
  "history": [
    {
      "content": "How is the weather in Redmond, US? And what's 2 + 2 + 1"

If tool calls are required by the LLM, instantiated objects of provided `LLMTool`s will be created at the `.tool_calls`:

In [7]:
response_0.tool_calls

[WeatherForecastTool(call_id='call_Y0RsmenHwmwo19qM7xngaVEY', location='Redmond, US', date=None, name='WeatherForecastTool', description='Call this to get the weather forecast for a particular location.'),
 AddNumbersTool(call_id='call_CDSTpjUMG6TFI7Uyn9e2o5z1', numbers=[2, 2, 1], name='add_numbers', description='Use this to sum a list of number.')]

Sending to the LLM the reply back with tool call result:

In [8]:
replies = [
    OpenAIChatRole.Tool.message(
        str(await tool.execute()), tool_call_id=tool.call_id or ""
    )
    for tool in response_0.tool_calls
]
replies

[{'content': 'Weather for Redmond, US(now) - nice weather.',
  'tool_call_id': 'call_Y0RsmenHwmwo19qM7xngaVEY',
  'role': 'tool'},
 {'content': '5',
  'tool_call_id': 'call_CDSTpjUMG6TFI7Uyn9e2o5z1',
  'role': 'tool'}]

In [9]:
response_1 = await chat_client(
    None,
    history=[*response_0.history, *replies],
    tools=[
        WeatherForecastTool,
        AddNumbersTool,
    ],
)

[INFO 2024-08-08 09:26:50,337 CHAT] - limit acquired for request, request_tokens=459, post_request_tokens=0
[INFO 2024-08-08 09:26:51,045 CHAT] - LLM usage with 251 total tokens (input=226, output=25)
[INFO 2024-08-08 09:26:51,045 CHAT] - limit released for request, request_tokens=459, post_request_tokens=0
[INFO 2024-08-08 09:26:51,046 CHAT] - request succeed with 0 retries in 0.71s and used 251 tokens


In [10]:
print(response_1.model_dump_json(indent=2))

{
  "output": {
    "raw_input": null,
    "raw_output": {
      "content": "- The weather in Redmond, US is nice.\n- 2 + 2 + 1 equals 5.",
      "role": "assistant",
      "function_call": null,
      "tool_calls": null
    },
    "content": "- The weather in Redmond, US is nice.\n- 2 + 2 + 1 equals 5.",
    "usage": {
      "input_tokens": 226,
      "output_tokens": 25,
      "total_tokens": 251
    }
  },
  "raw_json": null,
  "parsed_json": null,
  "history": [
    {
      "content": "How is the weather in Redmond, US? And what's 2 + 2 + 1",
      "role": "user"
    },
    {
      "role": "assistant",
      "content": null,
      "tool_calls": [
        {
          "id": "call_Y0RsmenHwmwo19qM7xngaVEY",
          "function": {
            "arguments": "{\"location\": \"Redmond, US\"}",
            "name": "WeatherForecastTool"
          },
          "type": "function"
        },
        {
          "id": "call_CDSTpjUMG6TFI7Uyn9e2o5z1",
          "function": {
            "argumen

In [11]:
print(response_1.output.content)

- The weather in Redmond, US is nice.
- 2 + 2 + 1 equals 5.
